In [1]:
from nlprocessing import ProcessCorpus
import pandas as pd
from nltk.corpus import words
import gensim

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [2]:
data1 = pd.read_csv('../data/comments.csv', sep='|').dropna()
data2 = pd.read_csv('../data/more_comments.csv').dropna()
data = data1.append(data2, ignore_index=True)
data = data[~data.duplicated()]
data = data[data['subreddit'] != 'politics']
test = data['text'].sample(n=10000)

In [3]:
corpus = ProcessCorpus()
corpus.fit(test, frequency=2)

In [9]:
for i in corpus.corpus_filtered:
    for j in i:
        if j == 'like':
            print('like')

In [4]:
[comment for comment in corpus.gensim_corpus_filtered if comment != []]

[[(1, 1)],
 [(13, 1)],
 [(21, 1)],
 [(29, 1), (31, 1), (37, 1)],
 [(46, 1)],
 [(65, 1)],
 [(74, 1), (77, 1)],
 [(89, 1), (90, 1), (97, 1)],
 [(100, 1)],
 [(107, 1)],
 [(107, 1)],
 [(120, 1)],
 [(123, 1), (124, 1)],
 [(125, 1)],
 [(135, 1)],
 [(142, 1)],
 [(155, 1)],
 [(170, 1)],
 [(176, 1)],
 [(179, 1)],
 [(13, 1), (77, 1), (183, 1)],
 [(195, 1), (198, 1), (208, 1)],
 [(223, 1), (229, 1), (238, 2), (240, 1), (250, 1)],
 [(46, 1), (261, 1)],
 [(264, 1)],
 [(265, 1)],
 [(276, 1)],
 [(284, 1)],
 [(289, 1)],
 [(291, 1)],
 [(284, 1)],
 [(300, 1)],
 [(303, 1)],
 [(284, 1), (319, 1)],
 [(195, 1), (324, 1)],
 [(343, 1)],
 [(349, 1), (350, 1)],
 [(355, 1)],
 [(77, 1), (360, 1), (361, 1), (367, 1)],
 [(183, 1)],
 [(379, 1)],
 [(390, 1)],
 [(398, 1)],
 [(408, 1)],
 [(428, 1)],
 [(432, 1), (433, 1)],
 [(1, 1), (435, 1)],
 [(439, 1)],
 [(46, 1), (442, 1)],
 [(183, 1), (428, 1), (450, 1)],
 [(176, 1), (467, 1), (476, 2), (478, 1), (481, 1)],
 [(284, 1), (490, 1), (496, 2), (503, 1), (504, 2), (508, 

In [25]:
model = gensim.models.LdaModel(
    corpus=corpus.gensim_corpus_filtered,
    id2word=corpus.gensim_dictionary,
    num_topics=5,
    update_every=1,
    chunksize=100,
    passes=10,
    alpha='auto',
    per_word_topics=True
)

In [33]:
coherence = gensim.models.CoherenceModel(
    model=model,
    texts=corpus.corpus_filtered,
    corpus=corpus.gensim_corpus_filtered
)
coherence.get_coherence()

0.5755995285087667

In [36]:
def find_perplexity_coherence(n_topics):
    model = gensim.models.LdaModel(
        corpus=corpus.gensim_corpus_filtered,
        id2word=corpus.gensim_dictionary,
        num_topics=5,
        update_every=1,
        chunksize=100,
        passes=10,
        alpha='auto',
        per_word_topics=True
    )
    coherence = gensim.models.CoherenceModel(
        model=model,
        texts=corpus.corpus_filtered,
        corpus=corpus.gensim_corpus_filtered
    )
    return [n_topics, model.log_perplexity(corpus.gensim_corpus_filtered), coherence.get_coherence()]

In [41]:
optimization = []
for i in range(20,32,2):
    optimization.append(find_perplexity_coherence(i))
optimization

[[20, -7.8239480355989075, 0.5912482399871739],
 [22, -7.833763480892517, 0.5841896704168112],
 [24, -7.83101400146077, 0.5972759201660975],
 [26, -7.837975234461614, 0.6055011334720193],
 [28, -7.844886814728603, 0.6241151163076838],
 [30, -7.842480232454846, 0.6185333114742362]]

In [40]:
optimization1=optimization.copy()

In [26]:
# model = gensim.models.LdaModel.load('mdl/model2/full_corpus_model')

In [27]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(model, corpus.gensim_corpus, corpus.gensim_dictionary)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3      0.028872 -0.065991       1        1  24.027772
0      0.033400 -0.061689       2        1  22.508104
2      0.020421 -0.150770       3        1  18.303515
1     -0.233692  0.090986       4        1  18.123539
4      0.150999  0.187465       5        1  17.037070, topic_info=           Term         Freq        Total Category  logprob  loglift
77          lol  2609.000000  2609.000000  Default  30.0000  30.0000
183      crypto  1392.000000  1392.000000  Default  29.0000  29.0000
170         eth  1229.000000  1229.000000  Default  28.0000  28.0000
1093        gme  1512.000000  1512.000000  Default  27.0000  27.0000
176         etc  1162.000000  1162.000000  Default  26.0000  26.0000
...         ...          ...          ...      ...      ...      ...
31    purchased   101.683998   106.798310   Topic5  -5.3586   1.7207
4954         ac    96.620011   101.801386   Topic5  -5.4097   1.7175
3038      loses    95.880978   101.031299   Topic5  -5.4174   1.7175
4281      april    92.259055    97.398588   Topic5  -5.4559   1.7156
2670       ebay    92.923450    98.224858   Topic5  -5.4487   1.7143

[187 rows x 6 columns], token_table=      Topic      Freq Term
term                      
4954      1  0.019646   ac
4954      2  0.019646   ac
4954      3  0.009823   ac
4954      4  0.009823   ac
4954      5  0.952836   ac
...     ...       ...  ...
1838      1  0.984871    🚀
1838      2  0.003444    🚀
1838      3  0.003444    🚀
1838      4  0.003444    🚀
1838      5  0.003444    🚀

[785 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 1, 3, 2, 5])